# ETRA Challenge Report

# Setup

In [10]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import missingno as msno

# classes for special types
from pandas.api.types import CategoricalDtype
from scipy.sparse._data import _data_matrix

# Apply the default theme
sns.set_theme()

In [11]:
# @formatter:off
%matplotlib inline
# should enable plotting without explicit call .show()

%load_ext pretty_jupyter
#@formatter:on

The pretty_jupyter extension is already loaded. To reload it, use:
  %reload_ext pretty_jupyter


## Downloading data

Download zip file from the web page. Description of the dataset is on the page [ETRA dataset description](https://etra.acm.org/2019/challenge.html)

In [12]:
from etra import ETRA

dataset = ETRA()

Dataset etra already downloaded.
Unpacking etra...


The directory ``data`` should therefore contain following directories/files

- data
- images
- DataSummary.csv

## Load Data

In [13]:
from etra import read_data

subject_no = 9
fix_puzzle_files = (dataset.data_dir / "data" / "{0:0>3}".format(subject_no)).glob("*Fixation_Puzzle_*.csv")
df_fix_puzzle = pd.concat((read_data(f) for f in fix_puzzle_files)).sort_values(by="Time")

fix_waldo_files = (dataset.data_dir / "data" / "{0:0>3}".format(subject_no)).glob("*Fixation_Waldo_*.csv")
df_fix_waldo = pd.concat((read_data(f) for f in fix_waldo_files)).sort_values(by="Time")

In [14]:
df_fix_puzzle.head()

,participant_id,trial_id,fv_fixation,task_type,stimulus_id,Time,LXpix,LYpix,RXpix,RYpix,LXhref,LYhref,RXhref,RYhref,LP,RP
0,009,012,Fixation,Puzzle,puz013,1034834,467.22,301.350,471.38,302.700,482.0,1228.0,527.0,1239.0,833,940
1,009,012,Fixation,Puzzle,puz013,1034836,460.82,304.350,464.82,304.950,487.0,1229.0,530.0,1233.0,830,943
2,009,012,Fixation,Puzzle,puz013,1034838,460.98,303.375,463.70,306.000,488.0,1219.0,519.0,1246.0,831,942
3,009,012,Fixation,Puzzle,puz013,1034840,461.30,302.775,465.86,306.225,491.0,1211.0,542.0,1246.0,828,950
4,009,012,Fixation,Puzzle,puz013,1034842,460.74,302.400,464.82,306.000,485.0,1208.0,531.0,1245.0,829,942


In [15]:
df_fix_waldo.head()

,participant_id,trial_id,fv_fixation,task_type,stimulus_id,Time,LXpix,LYpix,RXpix,RYpix,LXhref,LYhref,RXhref,RYhref,LP,RP
0,009,016,Fixation,Waldo,wal014,1614108,473.22,317.625,468.18,314.700,-802.0,2960.0,-858.0,2931.0,722,903
1,009,016,Fixation,Waldo,wal014,1614110,473.30,317.775,467.78,314.775,-801.0,2962.0,-862.0,2932.0,720,904
2,009,016,Fixation,Waldo,wal014,1614112,473.38,317.925,467.46,314.925,-800.0,2963.0,-866.0,2934.0,718,905
3,009,016,Fixation,Waldo,wal014,1614114,473.30,319.125,465.14,315.750,-800.0,2977.0,-889.0,2945.0,720,910
4,009,016,Fixation,Waldo,wal014,1614116,474.82,317.550,465.46,316.800,-785.0,2958.0,-886.0,2957.0,723,908


## Hypotheses



We will be interested in the following hypotheses:

1. **Is there a point where a participant becomes tired/bored when viewing a scene? (the frequency of saccades decreases) If so, are some scenes less interesting then others?**
    
    There are different types of scenes in the study and some are more visually stimulating then others. For example, it is not unreasonable to expect that a participant will stay much more engaged in a scene with a lot of detail to study then when looking at a blank scene. Another possibility is that during visual searches (picture puzzles, where is Waldo) a participant may become frustrated and just give up on the task. 

    The reasoning behind the way we are going to measure this is as follows. When a participant is engaged in a scene, we assume that their gaze is going to dart across the scene rapidly, as they are studying all the details and the number of saccades during a fixed time frame is going to be very high. On the other hand, when subject becomes bored or frustrated, we might expect that they are going to rest their gaze at some point and not move their eyes very much (maybe just expecting for the experiment to be over) and so the frequency of saccades is going to decrease.

    However, the decrease in frequency of saccades might not be exactly indicative of the subjects boredom or frustration. It is not unlikely that the frequency of saccades is high every time a new scene is introduced and naturally decreases as the subject becomes acquainted with the scene.

    Whatever the reason might be, if there indeed is such a changing point in the frequency of saccades, it might be interesting to study the time for such change and it's relationship to participant, scene and task.


2. **Does the pupil size change depending on scene? What about dependence on task?**

    

    



## Data manipulation

This is an optional section, in which you could describe, what did you do with the data to obtain given format. In our example, we just need to merge the data. Additionally, we want to detect fixations

In [16]:
from etra import detect

df_hyp1_samples = pd.concat([df_fix_puzzle, df_fix_waldo]).rename(
    {"Time": "time", "trial_id": "trial", "LXpix": "x", "LYpix": "y"}, axis=1)
df_hyp1_samples["time"] = df_hyp1_samples.groupby(["participant_id", "trial"])["time"].transform(lambda x: x - x.min())

df_hyp1_fix = []
groups = df_hyp1_samples.groupby(["participant_id", "trial"])
for (pid, trial), group in groups:
    tmp = detect(group)
    tmp = tmp[tmp["label"] == "FIXA"]
    tmp["participant_id"] = pid
    tmp["trial"] = trial
    df_hyp1_fix.append(tmp)

df_hyp1_fix = pd.concat(df_hyp1_fix)
df_hyp1_fix = df_hyp1_samples[
    ["participant_id", "trial", "fv_fixation", "task_type", "stimulus_id"]].drop_duplicates().merge(df_hyp1_fix, on=[
    "participant_id", "trial"], how="left")

Computed velocity exceeds threshold. Inappropriate filter setup? [1035.5 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1012.9 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1123.1 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1261.4 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1211.3 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1149.7 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1256.1 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1151.3 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1112.0 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1042.8 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1011.1 > 1000.0 deg/s]
Computed velocity exc

In [17]:
df_hyp1_fix.head()

,participant_id,trial,fv_fixation,task_type,stimulus_id,label,start_time,end_time,start_x,start_y,end_x,end_y,amp,peak_vel,med_vel,avg_vel
0,009,012,Fixation,Puzzle,puz013,FIXA,0,200,465.155030,302.178636,463.539654,305.027922,0.123688,34.836336,4.759935,5.555025
1,009,012,Fixation,Puzzle,puz013,FIXA,214,300,461.330476,311.542208,466.051515,312.529221,0.182137,24.240037,5.594435,7.583646
2,009,012,Fixation,Puzzle,puz013,FIXA,540,706,472.500346,316.706169,473.041991,315.633117,0.045392,20.638228,5.955275,6.629424
3,009,012,Fixation,Puzzle,puz013,FIXA,716,1044,474.261732,313.289935,475.944156,309.456494,0.158092,17.579624,4.784337,5.618096
4,009,012,Fixation,Puzzle,puz013,FIXA,1068,1142,486.907273,310.399026,479.285108,308.445779,0.297139,17.923089,7.603699,7.963162


## Results

In this section, describe statistical test that you used for testing your hypotheses. In general, the selection of statistical test depends on the type of variable.

There are following types of variables:

* Continuous - when the variable behaves as number. All fixation durations, pupil size, time are continuous variables
* Ordinal - variables do not behave as numbers, but you can order them. Grades in school are typical example. You can't say how many times is 1 better than 2, but you can say that 1 is better grade than 2. There are no nominal variables in this dataset, so I added this description only for completeness.
* Nominal - when variables are qualitative different. Type of task is an example of nominal variable

In the case of two variables, there are following options
* both variables continuous - regression or correlation (in R function `lm()` or `cor`/`cor.test` )
* both variables nominal - contingency tables a chi square test
* one variable nominal, other continuous - this is very common, this type is used, when we compare differences two condition - in this case, we use t.tests

There are three main types of t.tests

* Independent t-test - both groups contain independent data points (each data point is a different entity)
* Paired t-test - data points are linked to each other. This is typical example, when we measure same subjects multiple times
* One sample t-test - we are testing the sample against some theoretically interesting number

In case of more than two variables, we need to use ANOVAs. Usually, one variable will be dependent (outcome, the one which levels interest us) and others will be independent (predictors, the one, that we manipulate).
* For more than two groups, we use between-subject ANOVA
* For more than two measurement of same subject, we use within-subject ANOVA
* We can combine multiple between- and within-subject factors into mixed ANOVA

In our case, the simplest way how to test that is to aggregate data per each trial and use t-tests

### using t-tests

Because we have multiple data points from each participant, we can first aggregate data for each trial

In [18]:
avg_durations = df_hyp1_fix.assign(dur=lambda x: x.end_time - x.start_time)\
    .groupby(["task_type", "trial"])\
    .agg(avg_dur=("dur", "mean"))\
    .reset_index()
avg_durations

,task_type,trial,avg_dur
0,Puzzle,012,733.389831
1,Puzzle,022,605.314286
2,Puzzle,023,487.146067
3,Puzzle,032,619.742857
4,Puzzle,036,425.281250
5,Puzzle,043,641.261538
6,Puzzle,052,577.166667
7,Puzzle,061,666.625000
8,Puzzle,068,407.831579
9,Puzzle,079,461.662921


In [19]:
ttest_result = stats.ttest_rel(
    avg_durations[avg_durations.task_type == "Puzzle"].avg_dur,
    avg_durations[avg_durations.task_type == "Waldo"].avg_dur,
)

In [20]:
%%jinja markdown

Results show that there is no difference between Fixation duration in Where is Waldo and Puzzle (pvalue={{"{:.3}".format(ttest_result.pvalue)}})


Results show that there is no difference between Fixation duration in Where is Waldo and Puzzle (pvalue=0.541)